In [2]:
# imports and link xml
import re
from pprint import pprint
import xml.etree.ElementTree as ET
tree = ET.parse('session/Kismet-20191011-18-31-55-1.netxml')
root = tree.getroot()

In [3]:
StationList = [] # MAC, [[min GPS],[max GPS]], [[first seen],[last seen]], device type, encryption, ESSID

In [4]:
def getDeviceType(bssid): # find vendor based off MAC address
    file = open('mac-vendors.txt', 'r', encoding="utf8")
    raw=[]
    mac_dict={}
    for line in file:
        raw.append(re.split(r'\t+', line.strip()))
    for pair in raw:
        mac_dict[pair[0]]= pair[1]
    return mac_dict.get(bssid[:8])

In [5]:
def appendArray(net, child): # parse xml and create array data
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        if (net=="wireless-network"): # network xml tag
            scan= 'BSSID'
        else:
            scan= 'client-mac'        # client xml tag
        for bssid_tag in child.findall(scan):
            bssid = bssid_tag.text
        for gps_tag in child.findall("gps-info"):
            gps= [[gps_tag[0].text, gps_tag[1].text],[gps_tag[4].text, gps_tag[5].text]]
        time = [child.attrib['first-time'],child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'):
                essid= essid_tag.text
            for info_tag in ssid_tag :
                if (info_tag.tag == "encryption"):
                    encryption.append(info_tag.text)
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid]) 

In [6]:
for child in root:
    appendArray("wireless-network",child) #networks
    for subchild in child:
        appendArray("wireless-client", subchild) #clients
#print(StationList)